**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

In [ ]:
!sudo add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!sudo apt-get update -qq 2>&1 > /dev/null
!sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null
!google-drive-ocamlfuse

In [ ]:
!sudo apt-get install -qq w3m # to act as web browser 
!xdg-settings set default-web-browser w3m.desktop # to set default browser
%cd /content
!mkdir drive
%cd drive
!mkdir MyDrive
%cd ..
%cd ..
!google-drive-ocamlfuse /content/drive/MyDrive

**1) Clone the YoloR**



In [ ]:
!git clone https://github.com/WongKinYiu/yolor
%cd yolor

**2) Install Necessary Dependencies**




In [ ]:
!pip install -qr requirements.txt

!git clone https://github.com/JunnYu/mish-cuda
%cd mish-cuda
!git reset --hard 6f38976064cbcc4782f4212d7c0c5f6dd5e315a8
!python setup.py build install
%cd ..

!git clone https://github.com/fbcotter/pytorch_wavelets
%cd pytorch_wavelets
!pip install .
%cd ..

**3) Configure for Training YOLOR**

**NOTE:**

FILTER = (c+5)*3

max_batches = c*2000 (unless it's smaller than 6000)

steps = %80, %90*max_batches


In [ ]:
%cd yolor
!cp cfg/yolor_p6.cfg cfg/yolo_training.cfg

In [ ]:
!sed -i 's/subdivisions=8/subdivisions=8/' cfg/yolo_training.cfg
!sed -i 's/max_batches = 500500/max_batches = 6000/' cfg/yolo_training.cfg
!sed -i 's/steps=400000,450000/steps=4800,5400/' cfg/yolo_training.cfg

!sed -i '1614 s@classes=80@classes=1@' cfg/yolo_training.cfg
!sed -i '1658 s@classes=80@classes=1@' cfg/yolo_training.cfg
!sed -i '1702 s@classes=80@classes=1@' cfg/yolo_training.cfg
!sed -i '1746 s@classes=80@classes=1@' cfg/yolo_training.cfg

!sed -i '1605 s@filters=255@filters=18@' cfg/yolo_training.cfg
!sed -i '1649 s@filters=255@filters=18@' cfg/yolo_training.cfg
!sed -i '1693 s@filters=255@filters=18@' cfg/yolo_training.cfg
!sed -i '1737 s@filters=255@filters=18@' cfg/yolo_training.cfg

!sed -i '4 s@width=1280@width=416@' cfg/yolo_training.cfg
!sed -i '5 s@height=1280@height=416@' cfg/yolo_training.cfg

In [ ]:
!> /content/yolor/yolor.yaml
!echo -e "train: /content/images\nnc: 1\nnames: ['Car']" > /content/yolor.yaml

**4) Prepare Dataset**

In [ ]:
!mkdir /content/images/

import glob, os
zip_list = glob.glob("/content/drive/MyDrive/dataset/car_test/*.zip")
rar_list = glob.glob("/content/drive/MyDrive/dataset/car_test/*.rar")

for zip in zip_list:
  print(zip, " zip")
  os.system("unzip {} -d /content/images/".format(zip))

for rar in rar_list:
  print(rar, " rar")
  os.system("unrar e {} /content/images/".format(rar))

**5) Pretrain**

In [ ]:
%cd /content/yolor
!bash scripts/get_pretrain.sh

In [ ]:
%pip install -q wandb
import wandb
wandb.login()



**6) Start the Training**

In [ ]:
%cd /content/yolor
!python train.py --batch-size 1 --img 416 416 --data '/content/yolor/yolor.yaml' --cfg /content/yolor/yolo_training.cfg --weights '/content/yolor/yolor_p6.pt' --device 0 --name yolor_p6 --hyp '/content/yolor/data/hyp.scratch.1280.yaml' --epochs 30